## Intcode

In [1]:
############## IntCode ##############################
# states are
# 0 - running
# 1 - waiting for input
# 99 - halt

mutable struct Prog
    code::Vector{Int}
    cur::Int
    input::Vector{Int}
    output::Vector{Int}
    relative_base::Int
    state::Int
end

str2prog(s) = parse.(Int, split(s, ","))

Prog(s::String) = Prog(vcat(str2prog(s), zeros(Int, 10000)), 1, [], [], 0, 0)

copy_code(p::Prog) = Prog(copy(p.code), 1, [], [], 0, 0)

struct Instruction
    op::Int
    modes::Vector{Int}
end

function Instruction(op_code::Int)
    ops = Dict{Int, Int}(
        1 => 3,
        2 => 3,
        3 => 1,
        4 => 1,
        5 => 2,
        6 => 2,
        7 => 3,
        8 => 3,
        9 => 1,
        99 => 0
    )
    
    op = mod(op_code, 100)
    modes_code = div(op_code, 100)
    modes = zeros(ops[op])
    for i in 1:ops[op]
        modes[i] = mod(modes_code, 10)
        modes_code = div(modes_code, 10)
    end
    
    Instruction(op, modes)
end

Instruction(prog::Prog) = Instruction(prog.code[prog.cur])

function take(prog::Prog, mode, offset)
    if mode == 0
        return prog.code[prog.code[prog.cur + offset] + 1]
    elseif mode == 1
        return prog.code[prog.cur + offset]
    else
        return prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1]
    end
end

function update!(prog::Prog, value, offset, mode = 0)
    if mode == 0
        prog.code[prog.code[prog.cur + offset] + 1] = value
    elseif mode == 2
        prog.code[prog.code[prog.cur + offset] + prog.relative_base + 1] = value
    end
end

function apply(prog::Prog, instruction::Instruction)
    if instruction.op == 99
        prog.state = 99
    elseif instruction.op == 1
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 + a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 2
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 * a2, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 3
        if isempty(prog.input)
            prog.state = 1
        else
            update!(prog, popfirst!(prog.input), 1, instruction.modes[1])
            prog.cur += 2
        end
    elseif instruction.op == 4
        push!(prog.output, take(prog, instruction.modes[1], 1))
        prog.cur += 2
    elseif instruction.op == 5
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 != 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 6
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        prog.cur = a1 == 0 ? a2 + 1 : prog.cur + 3
    elseif instruction.op == 7
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 < a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 8
        a1 = take(prog, instruction.modes[1], 1)
        a2 = take(prog, instruction.modes[2], 2)
        update!(prog, a1 == a2 ? 1 : 0, 3, instruction.modes[3])
        prog.cur += 4
    elseif instruction.op == 9
        prog.relative_base += take(prog, instruction.modes[1], 1)
        prog.cur += 2
    end
end

function run(prog::Prog, input::Vector{Int})
    prog.input = input
    prog.state = 0
    run(prog)
end

run(prog::Prog, input::Int) = run(prog, [input])

function run(prog)
    clear!(prog)
    while prog.state == 0
        instruction = Instruction(prog)
        apply(prog, instruction)
    end
    
    prog.output
end

function feed(prog::Prog, input::Vector{Int})
    prog.input = vcat(prog.input, input)
    prog.state = 0
end

clear!(prog) = (prog.output = [])

clear! (generic function with 1 method)

In [2]:
vm = Prog(readline("input.txt"))

Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 1, Int64[], Int64[], 0, 0)

In [12]:
struct Packet
    from::Int
    to::Int
    x::Int
    y::Int
end

process_output(output::Vector{T}, id::Int) where T = 
    [Packet(id, output[3*i + 1], output[3*i + 2], output[3*i + 3]) for i in 0:(div(length(output), 3) - 1)]

struct Network
    vms::Dict{Int, Prog}
    packets::Vector{Packet}
end

function Network()
    vms = Dict{Int, Prog}()
    packets = Packet[]
    for i in 0:49
        vm = Prog(readline("input.txt"))
        run(vm, i)
        vms[i] = vm
        out = run(vm, -1)
        append!(packets, process_output(out, i))
    end
    
    Network(vms, packets)
end

function tick(nw::Network)
    if isempty(nw.packets)
        for (k, vm) in nw.vms
            out = run(vm, -1)
            append!(vm.packets, process_output(out, k))
        end
    else
        while !isempty(nw.packets)
            packet = pop!(nw.packets)
            out = run(nw.vms[packet.to], [packet.x, packet.y])
            append!(vm.packets, process_output(out, packet.to))
        end
    end
end

tick (generic function with 1 method)

In [13]:
network = Network()

Network(Dict(2 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], [39, 85451, -173], 2237, 1),11 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], Int64[], 2237, 1),39 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], Int64[], 2237, 1),46 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], [27, 99556, 257], 2237, 1),25 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], [14, 158889, -186], 2237, 1),42 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], Int64[], 2237, 1),29 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], Int64[], 2237, 1),8 => Prog([3, 62, 1001, 62, 11, 10, 109, 2237, 105, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 74, Int64[], Int64[], 

In [15]:
tick(network)

ErrorException: type Prog has no field packets

In [10]:
for i in 0:49
    vm = Prog(readline("input.txt"))
    println(run(vm, i))
    println(run(vm, -1))
end

Int64[]
[10, 71342, 29058, 47, 166142, 29058, 47, 249213, 29058, 20, 36373, 29058, 20, 72746, 29058, 20, 109119, 29058]
Int64[]
Int64[]
Int64[]
[39, 85451, -173]
Int64[]
[1, 299577, -31]
Int64[]
Int64[]
Int64[]
[34, 71931, 821]
Int64[]
Int64[]
Int64[]
[8, 96293, -671]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
[34, 23977, 1217]
Int64[]
[21, 356765, 160]
Int64[]
Int64[]
Int64[]
[4, 164319, 125]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
[44, 57734, 2447]
Int64[]
[4, 109546, 5, 4, 219092, 5, 21, 142706, 5]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
[34, 95908, 3, 27, 24889, 3, 27, 74667, 3, 30, 46559, 3]
Int64[]
[34, 47954, 1, 39, 170902, 1, 8, 192586, 1, 44, 86601, 1, 14, 52963, 1]
Int64[]
[8, 288879, 1061]
Int64[]
[14, 158889, -186]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
[30, 93118, 7890373]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
Int64[]
[27, 49778, -1863]
Int64[]
Int64[]
Int64[]
[6, 17989, 9839]
Int64[]
[4, 54773, 10, 21, 428118, 10]
In